In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-29"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12417,2024-02-29,Eua Nba,00:00,Los Angeles Clippers,Los Angeles Lakers,1.63,2.36,233.5,1.88,2.04,-3.5,1.97,2.90,QVqudS0S,0.613497,0.423729,0.531915,0.490196,0.037226,0.4,0.6,NaN,NaN,178.870,54.703678,0.305829,228.176,75.721761,0.331857,148.260,269.868,153.00,272.33,0.0,0.0,0.0,0.0,0.258741,0.057723,0.270065,-1.39,-0.278,-2.266187,0.744021,0.8,0.055979,0.98,0.196,6.938776,0.432265,0.4,-0.032265
12418,2024-02-29,Europa Euroliga,16:00,Alba Berlin,Maccabi Tel Aviv,2.99,1.42,172.5,1.91,1.91,4.5,2.00,1.18,MsHea8Q4,0.334448,0.704225,0.523560,0.523560,0.038674,0.4,0.8,NaN,NaN,356.908,260.989521,0.731252,132.016,55.363024,0.419366,189.610,214.058,150.48,224.77,0.0,0.0,0.0,0.0,0.503473,0.000000,0.364671,-2.36,-0.472,-4.216102,0.394792,0.3,-0.094792,-2.21,-0.442,-0.950226,0.505601,0.3,-0.205601
12419,2024-02-29,Europa Euroliga,16:45,Real Madrid,Panathinaikos,1.34,3.37,163.5,1.91,1.91,-7.5,1.95,7.00,fX88d6AN,0.746269,0.296736,0.523560,0.523560,0.043005,0.8,0.2,NaN,NaN,115.844,19.261035,0.166267,121.732,13.898333,0.114172,119.536,139.420,92.02,117.64,0.0,0.0,0.0,0.0,0.609523,0.000000,0.797964,0.86,0.172,1.976744,0.855509,1.0,0.144491,-1.25,-0.250,-9.480000,0.550094,0.5,-0.050094
12420,2024-02-29,Europa Bibl,15:00,Peja,Sigal Prishtina,1.77,1.95,157.5,1.81,1.85,-2.5,2.04,2.35,Y1IUY9Vg,0.564972,0.512821,0.552486,0.540541,0.077792,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,124.32,151.89,0.0,0.0,0.0,0.0,0.068430,0.015456,0.099865,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12421,2024-02-29,Europa Eurocopa Feminina,16:00,Lattes Montpellier F,Besiktas F,2.33,1.55,144.5,1.81,1.85,-2.5,2.05,1.76,SbIKkJOl,0.429185,0.645161,0.552486,0.540541,0.074346,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,97.17,308.56,0.0,0.0,0.0,0.0,0.284301,0.015456,0.107644,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12422,2024-02-29,Polônia Liga Feminina De Basquete,16:00,Lublin F,Arka Gdynia F,2.04,1.73,142.5,1.87,1.87,1.5,1.91,1.56,KbUqAzHO,0.490196,0.578035,0.534759,0.534759,0.068231,0.2,0.8,NaN,NaN,408.192,644.106076,1.577949,142.238,50.703232,0.356468,104.978,172.576,93.06,93.93,0.0,0.0,0.0,0.0,0.116288,0.000000,0.142644,-2.21,-0.442,-2.352941,0.000000,0.0,0.000000,2.04,0.408,1.789216,0.700318,0.8,0.099682
12423,2024-02-29,Eua Ncaa,21:00,Albany,New Hampshire,1.80,2.00,163.5,1.80,1.91,-1.5,1.91,2.00,l6W4iwKn,0.555556,0.500000,0.555556,0.523560,0.055556,0.8,0.2,NaN,NaN,165.316,74.596497,0.451236,209.566,131.210764,0.626107,154.704,208.076,108.73,219.24,0.0,0.0,0.0,0.0,0.074432,0.041931,0.032552,-2.13,-0.426,-1.877934,0.000000,0.0,0.000000,2.93,0.586,1.706485,0.410043,0.6,0.189957
12424,2024-02-29,Eua Ncaa,21:00,Drexel,Stony Brook,1.20,4.80,139.5,1.91,1.91,-10.5,1.95,11.00,vZF0pa6b,0.833333,0.208333,0.523560,0.523560,0.041667,0.8,0.6,NaN,NaN,120.382,25.035533,0.207967,134.370,51.481200,0.383130,99.086,178.272,92.34,125.51,0.0,0.0,0.0,0.0,0.848528,0.000000,0.988311,1.37,0.274,0.729927,0.787863,0.9,0.112137,-3.49,-0.698,-5.444126,0.344586,0.3,-0.044586
12425,2024-02-29,Eua Ncaa,21:00,Le Moyne,Wagner,1.83,1.91,131.5,1.83,1.83,-1.5,2.00,2.00,dloIVa25,0.546448,0.523560,0.546448,0.546448,0.070008,0.6,0.8,NaN,NaN,134.982,53.538248,0.396633,129.174,28.174969,0.218116,117.768,135.204,98.02,104.65,0.0,0.0,0.0,0.0,0.030251,0.000000,0.000000,1.15,0.230,3.608696,0.000000,0.0,0.000000,-1.21,-0.242,-3.760331,0.455463,0.5,0.044537
12426,2024

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Los Angeles Clippers,Los Angeles Lakers,233.5,1.88,1.0,Over
1,16:45,Europa Euroliga,Real Madrid,Panathinaikos,163.5,1.91,1.0,Over
2,15:00,Europa Bibl,Peja,Sigal Prishtina,157.5,1.81,1.0,Over
3,21:00,Eua Ncaa,Drexel,Stony Brook,139.5,1.91,1.0,Over
4,21:00,Eua Ncaa,Le Moyne,Wagner,131.5,1.83,1.0,Over
5,22:00,Eua Ncaa,North Dakota,South Dakota St.,149.5,1.91,1.0,Over
6,22:30,Eua Ncaa,Southern Indiana,Lindenwood,143.5,1.91,1.0,Over
7,22:30,Eua Ncaa,Southern Utah,Abilene Christian,153.5,1.91,1.0,Over
